In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import json
from collections import defaultdict
import pickle


In [7]:
root_dir = './save/output'

dirs = [f for f in os.listdir(root_dir) ]
dirs.sort()
print(dirs)

features_train = []
features_dev = []

all_results_train = []
all_results_dev = []

tokenizers_train = []
tokenizers_dev = []

for model_dir in dirs:
    model_name = ''
    tokenizer = None
    all_results = None
    features = None
    
    
    with open(os.path.join(root_dir, model_dir, 'config.json')) as f:
        config = json.load(f)
        model_name = config['model_name'].replace('-dev', '').replace('-train', '').replace('-01', '')
    
    print(model_name, config['type'])
    
    with open(os.path.join(root_dir, model_dir, 'features.pkl'), 'rb') as f:
        features = pickle.load(f)
        
    with open(os.path.join(root_dir, model_dir, 'all_results.pkl'), 'rb') as f:
        all_results = pickle.load(f)
    
            
    with open(os.path.join(root_dir, model_dir, 'tokenizer.pkl'), 'rb') as f:
        tokenizer = pickle.load(f)
        
    if config['type'] == 'train':
        features_train.append(features)
        all_results_train.append(all_results)
        tokenizers_train.append(tokenizer)
    elif config['type'] == 'dev':
        features_dev.append(features)
        all_results_dev.append(all_results)
        tokenizers_dev.append(tokenizer)
        

['albert-large-v2-dev', 'albert-large-v2-train', 'bert-base-dev', 'bert-base-train', 'bert-large-dev', 'bert-large-train', 'roberta-large-dev', 'roberta-large-train']
albert-large-v2 dev


/home/alvin/miniconda3/envs/squad/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/alvin/miniconda3/envs/squad/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/alvin/miniconda3/envs/squad/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/alvin/miniconda3/envs/squad/lib/python3.6/s

albert-large-v2 train
bert-base dev
bert-base train
bert-large dev
bert-large train
roberta-large dev
roberta-large train


In [8]:
# bewared of OOM issues
# train_to_save = [features_train, all_results_train, tokenizers_train]
with open(os.path.join(root_dir, 'saved_data_train.pkl'), 'wb') as f:
    pickle.dump([features_train, all_results_train, tokenizers_train], f)

In [9]:
del train_to_save 

NameError: name 'train_to_save' is not defined

In [10]:
dev_to_save = [features_dev, all_results_dev, tokenizers_dev]
with open(os.path.join(root_dir, 'saved_data_dev.pkl'), 'wb') as f:
    pickle.dump(dev_to_save, f)

## Output for Pure Start End Scores

In [3]:

dir = './runs'
directory = os.fsencode('./runs')

names = []
dfs = []
for file in os.listdir(directory):
    filename = os.fsdecode(file)
    df = pd.read_csv(os.path.join(dir, filename))
    dfs.append(df)
    names.append(filename)


In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(14, 6)
ax.set(xlabel='Steps', ylabel='Eval F1', title='Eval F1 vs. steps')
ax.grid()

for idx, df in enumerate(dfs):
    label = names[idx].split('-0')[0][4:]
    plt.plot(df['Step'], df['Value'], label=label)
ax.legend()
axes = plt.gca()
plt.yticks(np.arange(64, 92, 2))
# fig.savefig("eval_f1_runs.png")

In [2]:
root_dir = './save/output'
directory = os.fsencode(root_dir)

d = defaultdict(defaultdict)

for dir in os.listdir(directory):
    model_dir = os.fsdecode(dir)
    with open(os.path.join(root_dir, model_dir, 'model_output.json')) as f:
        x = json.load(f)
        model_name = x['model_name'].replace('-dev', '').replace('-train', '').replace('-01', '')
        d[model_name][x['type']] = np.array(x['output'])
        del x
    



In [3]:
train = []
dev = []
for k, v in d.items():
    train.append(v['train'])
    dev.append(v['dev'])
    print(k)
    
    


albert-xxlarge-v1
albert-large-v2
roberta-large
bert-large


In [4]:
np_train = np.array(train).transpose(1, 0, 2, 3)
np_train.shape

(130319, 4, 2, 256)

In [5]:
np_dev = np.array(dev).transpose(1, 0, 2, 3)
np_dev.shape

(6078, 4, 2, 256)

In [8]:
import h5py
with h5py.File(os.path.join(root_dir, 'model_output.h5'), 'w') as hf:
    hf.create_dataset("model_output_train",  data=np_train)
    hf.create_dataset("model_output_dev",  data=np_dev)

In [10]:
with h5py.File(os.path.join(root_dir, 'model_output.h5'), 'r') as hf:
    data_train = hf['model_output_train'][:]
    data_dev = hf['model_output_dev'][:]
    print(data_train.shape, data_dev.shape)

(130319, 4, 2, 256) (6078, 4, 2, 256)
